<a href="https://colab.research.google.com/github/Tar-ive/WIC_clustering/blob/main/wic_data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('RTPH_Survey_stand_TLL_4.1.25.csv')
print(f"Loaded dataset with {df.shape[0]} rows and {df.shape[1]} columns")

# Step 1: Map variable names to match our previous analysis
# ------------------------------------------------------------------
print("Mapping variable names...")

# Create dictionary to map new column names to the ones we used before
var_mapping = {
    'ResponseId': 'Response ID',                      # ID column
    'male_binary': 'Gender',                          # Demographics
    'poor_physhealth_binary': 'poor_phys',            # Health status
    'poorMH': 'poor_mental',
    'poor_QOL_binary': 'poor_qol',
    'sought_care_recently_binary': 'sought_care_recently',  # Outcomes
    'pct_times_got_care': 'pct_times_get_Care',
    'mean_satisfied_w_care': 'mean_satisfaction'
}

# Create new columns with the old names for consistency
for new_col, old_col in var_mapping.items():
    if new_col in df.columns:
        df[old_col] = df[new_col]
    else:
        print(f"Warning: {new_col} not found in the dataset")

# Special case: Gender was 1=Male, 0=Female in the original dataset
# In the new dataset it's male_binary (1=Male, 0=Female)
# Since we want 1=Female, 0=Male to match the previous analysis:
if 'Gender' in df.columns:
    df['Gender'] = 1 - df['Gender']  # Convert male_binary to female_binary

# Step 2: Create the intermediate variables needed for our composite measures
# ------------------------------------------------------------------
print("Creating intermediate variables...")

# Create age groups from age column
df['age_group'] = pd.cut(df['age'],
                         bins=[0, 44, 64, float('inf')],
                         labels=['18-44', '45-64', '65+'])

# Create health issues count
health_vars = ['poor_phys', 'poor_mental', 'poor_qol', 'chronic_cond_binary']
if all(var in df.columns for var in health_vars):
    df['health_issues_count'] = df[health_vars].sum(axis=1)
    print("Created health_issues_count")
else:
    missing = [var for var in health_vars if var not in df.columns]
    print(f"Cannot create health_issues_count - missing: {missing}")

# Create social determinants of health vulnerability score
sdoh_vars = ['food_insecure_binary', 'house_insecure_binary',
             'transport_insecure_binary', 'income_insecure_binary']
if all(var in df.columns for var in sdoh_vars):
    df['sdoh_vulnerability_score'] = df[sdoh_vars].sum(axis=1)
    print("Created sdoh_vulnerability_score")
else:
    missing = [var for var in sdoh_vars if var not in df.columns]
    print(f"Cannot create sdoh_vulnerability_score - missing: {missing}")

# Create facilities within 30 minutes measure
# Note: The new dataset has specific columns for each facility type
if all(col in df.columns for col in ['prim_care_less30min', 'er_care_less30min']):
    # Count how many facility types are within 30 minutes
    facility_cols = [col for col in df.columns if col.endswith('_less30min')]
    df['facilities_within_30min'] = df[facility_cols].sum(axis=1)
    print(f"Created facilities_within_30min from {len(facility_cols)} facility columns")
else:
    # Alternative: Use original Q2.19 columns to calculate if available
    proximity_cols = ['Q2.19_1', 'Q2.19_3', 'Q2.19_4', 'Q2.19_5']
    if all(col in df.columns for col in proximity_cols):
        df['facilities_within_30min'] = df[proximity_cols].apply(
            lambda x: sum([(1 if pd.notnull(val) and val in [1, 2] else 0) for val in x]), axis=1)
        print("Created facilities_within_30min from Q2.19 columns")
    else:
        print("Cannot create facilities_within_30min - missing required columns")

# Step 3: Create the exact variables needed for our focused dataset
# ------------------------------------------------------------------
print("Creating focused dataset variables...")

# 1. Create high_health_need indicator
if 'health_issues_count' in df.columns:
    df['high_health_need'] = (df['health_issues_count'] >= 2).astype(int)
    # Create healthcare needs binary
    df['high_needs_binary'] = ((df['high_health_need'] + df['chronic_cond_binary'] >= 1)
                              .astype(int))
    print("Created high_needs_binary")
else:
    print("Cannot create high_needs_binary - missing health_issues_count")

# 2. Create high_social_vulnerability indicator
if 'sdoh_vulnerability_score' in df.columns:
    df['high_social_vulnerability'] = (df['sdoh_vulnerability_score'] >= 2).astype(int)
    print("Created high_social_vulnerability")
else:
    print("Cannot create high_social_vulnerability - missing sdoh_vulnerability_score")

# 3. Create low_geographic_access indicator
if 'facilities_within_30min' in df.columns:
    df['low_geographic_access'] = (df['facilities_within_30min'] <= 2).astype(int)
    print("Created low_geographic_access")
else:
    print("Cannot create low_geographic_access - missing facilities_within_30min")

# 4. Create healthcare barriers binary (from high_social_vulnerability, low_geographic_access, self_insured_binary)
if all(var in df.columns for var in ['high_social_vulnerability', 'low_geographic_access', 'self_insured_binary']):
    df['healthcare_barriers'] = (
        df['high_social_vulnerability'] +
        df['low_geographic_access'] +
        (1 - df['self_insured_binary'])  # Reverse coding: 1 = not insured
    )
    df['high_barriers_binary'] = (df['healthcare_barriers'] >= 2).astype(int)
    print("Created high_barriers_binary")
else:
    print("Cannot create high_barriers_binary - missing required variables")

# 5. Create demographic group indicators
if all(var in df.columns for var in ['age_group', 'college_ed_binary']):
    # Create demographic group
    conditions = [
        (df['age_group'] == '18-44') & (df['college_ed_binary'] == 1),
        (df['age_group'] == '18-44') & (df['college_ed_binary'] == 0),
        df['age_group'].isin(['45-64', '65+'])
    ]
    choices = ['young_educated', 'young_uneducated', 'middle_older']
    df['demographic_group'] = np.select(conditions, choices, default='unknown')

    # Create boolean flags for each demographic group
    df['demo_young_educated'] = (df['demographic_group'] == 'young_educated')
    df['demo_young_uneducated'] = (df['demographic_group'] == 'young_uneducated')
    df['demo_middle_older'] = (df['demographic_group'] == 'middle_older')
    print("Created demographic group indicators")
else:
    print("Cannot create demographic groups - missing required variables")

# Convert percentage string to float for pct_times_get_Care if needed
if 'pct_times_get_Care' in df.columns and df['pct_times_get_Care'].dtype == 'object':
    df['pct_times_get_Care'] = df['pct_times_get_Care'].str.rstrip('%').astype('float') / 100
    print("Converted pct_times_get_Care to float")

# Step 4: Create the exact focused dataset with matching structure
# ------------------------------------------------------------------
print("Creating final focused dataset...")

# Select the exact columns needed for the focused dataset
focused_columns = [
    'Response ID',                 # Identifier
    'high_barriers_binary',        # Barriers
    'high_needs_binary',           # Health needs
    'demo_young_educated',         # Demographics
    'demo_young_uneducated',
    'demo_middle_older',
    'sought_care_recently',        # Outcomes
    'pct_times_get_Care',
    'mean_satisfaction'
]

# Check which columns we have
available_columns = [col for col in focused_columns if col in df.columns]
missing_columns = [col for col in focused_columns if col not in df.columns]
if missing_columns:
    print(f"Warning: Missing columns in final dataset: {missing_columns}")

# Create the focused dataset
focused_df = df[available_columns].copy()

# Ensure boolean columns are actually boolean type
for col in ['demo_young_educated', 'demo_young_uneducated', 'demo_middle_older']:
    if col in focused_df.columns:
        focused_df[col] = focused_df[col].astype('bool')

# Check for missing values
missing_values = focused_df.isnull().sum()
print("\nMissing values in focused dataset:")
print(missing_values[missing_values > 0])

# Create a version without missing values
focused_df_complete = focused_df.dropna()
print(f"\nFocused dataset: {focused_df.shape[0]} rows before removing missing values")
print(f"Focused dataset: {focused_df_complete.shape[0]} rows after removing missing values")

# Save the datasets
focused_df.to_csv('focused_dataset_new_all.csv', index=False)
focused_df_complete.to_csv('focused_dataset_new.csv', index=False)

print("\nFocused dataset created and saved as 'focused_dataset_new.csv'")
print("Sample of the focused dataset:")
print(focused_df.head())

# Compare data types with original focused dataset
print("\nColumn data types in new focused dataset:")
print(focused_df.dtypes)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('/content/189_dataset - NUMERIC_HealthIssuesRuralTX_Net.csv')

# Create age groups from Age column
df['age_group'] = pd.cut(pd.to_numeric(df['Age'], errors='coerce'),
                         bins=[0, 44, 64, float('inf')],
                         labels=['18-44', '45-64', '65+'])

# Create health issues count
df['health_issues_count'] = df['poor_phys'] + df['poor_mental'] + df['poor_qol'] + df['chronic_cond_binary']

# Create social determinants of health vulnerability score
df['sdoh_vulnerability_score'] = (df['food_insecure_binary'] + df['house_insecure_binary'] +
                                 df['transport_insecure_binary'] + df['income_insecure_binary'])

# Create healthcare proximity measures
# Count facilities within 30 minutes (assuming 1 and 2 are the codes for "within 30 minutes")
proximity_time_cols = ['Q2.19_1', 'Q2.19_2', 'Q2.19_3', 'Q2.19_4', 'Q2.19_5']
df['facilities_within_30min'] = df[proximity_time_cols].apply(
    lambda x: sum([(1 if pd.notnull(val) and val in [1, 2] else 0) for val in x]), axis=1)

# Count facilities within 10 miles (assuming 1 and 2 are the codes for "within 10 miles")
proximity_distance_cols = ['Q2.20_1', 'Q2.20_2', 'Q2.20_3', 'Q2.20_4', 'Q2.20_5']
df['facilities_within_10miles'] = df[proximity_distance_cols].apply(
    lambda x: sum([(1 if pd.notnull(val) and val in [1, 2] else 0) for val in x]), axis=1)

# Create binary indicators for specific facilities being nearby
df['primary_care_nearby'] = df['Q2.19_1'].apply(lambda x: 1 if pd.notnull(x) and x in [1, 2] else 0)
df['urgent_care_nearby'] = df['Q2.19_2'].apply(lambda x: 1 if pd.notnull(x) and x in [1, 2] else 0)
df['er_nearby'] = df['Q2.19_5'].apply(lambda x: 1 if pd.notnull(x) and x in [1, 2] else 0)

# Create incorrect care binary indicator from Q2.22
df['incorrect_care_binary'] = df['Q2.22'].apply(lambda x: 1 if pd.notnull(x) and x == 1 else 0)

# Create the final dataset for clustering with selected variables
cluster_df = df[['Response ID', 'age_group', 'Gender', 'college_ed_binary', 'employed_binary',
                 'chronic_cond_binary', 'pregnant_binary', 'poor_phys', 'poor_mental', 'poor_qol',
                 'health_issues_count', 'food_insecure_binary', 'house_insecure_binary',
                 'transport_insecure_binary', 'income_insecure_binary', 'sdoh_vulnerability_score',
                 'self_insured_binary', 'difficult_afford_health', 'facilities_within_30min',
                 'facilities_within_10miles', 'primary_care_nearby', 'urgent_care_nearby', 'er_nearby',
                 'difficulty_coping_binary', 'sought_care_recently', 'pct_times_get_Care',
                 'mean_satisfaction', 'incorrect_care_binary']].copy()

# Convert percentage string to float if needed
if 'pct_times_get_Care' in cluster_df.columns and cluster_df['pct_times_get_Care'].dtype == 'object':
    cluster_df['pct_times_get_Care'] = cluster_df['pct_times_get_Care'].str.rstrip('%').astype('float') / 100

# Handle missing values for branching logic questions without removing records
# For questions where NA means "not applicable" due to branching logic, fill with appropriate values

# For pregnant_binary, NA likely means not applicable (male respondent or didn't answer)
if 'pregnant_binary' in cluster_df.columns:
    cluster_df['pregnant_binary'] = cluster_df['pregnant_binary'].fillna(0)  # Not applicable = 0

# For pct_times_get_Care, if NA, it might mean they never needed care
# Rather than drop these records, we can set to a value that makes sense (e.g., 1.0 for 100% since they never failed)
if 'pct_times_get_Care' in cluster_df.columns:
    cluster_df['pct_times_get_Care'] = cluster_df['pct_times_get_Care'].fillna(1.0)

# For mean_satisfaction, if NA, might mean they never sought care
# Fill with the middle/neutral value of the scale (assuming it's a 1-5 scale)
if 'mean_satisfaction' in cluster_df.columns:
    cluster_df['mean_satisfaction'] = cluster_df['mean_satisfaction'].fillna(3.0)

# For incorrect_care_binary, NA might mean they haven't sought care
if 'incorrect_care_binary' in cluster_df.columns:
    cluster_df['incorrect_care_binary'] = cluster_df['incorrect_care_binary'].fillna(0)

# Check that we have all 159 records
print(f"Number of records in final dataset: {len(cluster_df)}")

# Create a standardized version for clustering
cluster_data_standardized = cluster_df.copy()

# One-hot encode categorical variables
cluster_data_standardized = pd.get_dummies(cluster_data_standardized,
                                          columns=['age_group'],
                                          drop_first=False)

# Standardize continuous variables
scaler = StandardScaler()
continuous_cols_to_scale = ['health_issues_count', 'sdoh_vulnerability_score',
                            'facilities_within_30min', 'facilities_within_10miles',
                            'pct_times_get_Care', 'mean_satisfaction']

# Create standardized versions with suffix
for col in continuous_cols_to_scale:
    if col in cluster_data_standardized.columns:
        new_col_name = f"{col}_scaled"
        cluster_data_standardized[new_col_name] = scaler.fit_transform(
            cluster_data_standardized[[col]])

# Save both versions
cluster_df.to_csv('cluster_data_.csv', index=False)
cluster_data_standardized.to_csv('cluster_data_standardized.csv', index=False)

# Display the first few rows of the prepared dataset
print("\nOriginal cluster data (first 5 rows):")
print(cluster_df.head())